# 第二讲：Python基础二

## 字符编码

我们已经讲过了，字符串也是一种数据类型，但是，字符串比较特殊的是还有一个编码问题。

因为计算机只能处理数字，如果要处理文本，就必须先把文本转换为数字才能处理。最早的计算机在设计时采用8个比特（bit）作为一个字节（byte），所以，一个字节能表示的最大的整数就是255（二进制11111111=十进制255），如果要表示更大的整数，就必须用更多的字节。比如两个字节可以表示的最大整数是65535，4个字节可以表示的最大整数是4294967295。

由于计算机是美国人发明的，因此，最早只有127个字符被编码到计算机里，也就是大小写英文字母、数字和一些符号，这个编码表被称为ASCII编码，比如大写字母A的编码是65，小写字母z的编码是122。

但是要处理中文显然一个字节是不够的，至少需要两个字节，而且还不能和ASCII编码冲突，所以，中国制定了GB2312编码，用来把中文编进去。

你可以想得到的是，全世界有上百种语言，日本把日文编到Shift_JIS里，韩国把韩文编到Euc-kr里，各国有各国的标准，就会不可避免地出现冲突，结果就是，在多语言混合的文本中，显示出来会有乱码。

因此，Unicode字符集应运而生。Unicode把所有语言都统一到一套编码里，这样就不会再有乱码问题了。

Unicode标准也在不断发展，但最常用的是UCS-16编码，用两个字节表示一个字符（如果要用到非常偏僻的字符，就需要4个字节）。现代操作系统和大多数编程语言都直接支持Unicode。

现在，捋一捋ASCII编码和Unicode编码的区别：ASCII编码是1个字节，而Unicode编码通常是2个字节。

字母A用ASCII编码是十进制的65，二进制的01000001；

字符0用ASCII编码是十进制的48，二进制的00110000，注意字符'0'和整数0是不同的；

汉字中已经超出了ASCII编码的范围，用Unicode编码是十进制的20013，二进制的01001110 00101101。

你可以猜测，如果把ASCII编码的A用Unicode编码，只需要在前面补0就可以，因此，A的Unicode编码是00000000 01000001。

新的问题又出现了：如果统一成Unicode编码，乱码问题从此消失了。但是，如果你写的文本基本上全部是英文的话，用Unicode编码比ASCII编码需要多一倍的存储空间，在存储和传输上就十分不划算。

所以，本着节约的精神，又出现了把Unicode编码转化为“可变长编码”的UTF-8编码。UTF-8编码把一个Unicode字符根据不同的数字大小编码成1-6个字节，常用的英文字母被编码成1个字节，汉字通常是3个字节，只有很生僻的字符才会被编码成4-6个字节。如果你要传输的文本包含大量英文字符，用UTF-8编码就能节省空间。

UTF-8编码有一个额外的好处，就是ASCII编码实际上可以被看成是UTF-8编码的一部分，所以，大量只支持ASCII编码的历史遗留软件可以在UTF-8编码下继续工作。

搞清楚了ASCII、Unicode和UTF-8的关系，我们就可以总结一下现在计算机系统通用的字符编码工作方式：

在计算机内存中，统一使用Unicode编码，当需要保存到硬盘或者需要传输的时候，就转换为UTF-8编码。

用记事本编辑的时候，从文件读取的UTF-8字符被转换为Unicode字符到内存里，编辑完成后，保存的时候再把Unicode转换为UTF-8保存到文件。

浏览网页的时候，服务器会把动态生成的Unicode内容转换为UTF-8再传输到浏览器。所以你看到很多网页的源码上会有类似<meta charset="UTF-8" />的信息，表示该网页正是用的UTF-8编码。



## Python的字符串

在最新的Python 3版本中，字符串是以Unicode编码的，也就是说，Python的字符串支持多语言，例如：

In [ ]:
print('包含中文的str')

对于单个字符的编码，Python提供了ord()函数获取字符的整数表示，chr()函数把编码转换为对应的字符：

In [1]:
print(ord('A'))
print(ord('中'))
print(chr(66))
print(chr(25991))

65
20013
B
文


如果知道字符的整数编码，还可以用十六进制这么写str:

In [2]:
print('\u4e2d\u6587')

中文


两种写法完全是等价的。

由于Python的字符串类型是str，在内存中以Unicode表示，一个字符对应若干个字节。如果要在网络上传输，或者保存到磁盘上，就需要把str变为以字节为单位的bytes。

Python对bytes类型的数据用带b前缀的单引号或双引号表示：

In [ ]:
x = b'ABC'

要注意区分'ABC'和b'ABC'，前者是str，后者虽然内容显示得和前者一样，但bytes的每个字符都只占用一个字节。

以Unicode表示的str通过encode()方法可以编码为指定的bytes，例如：

In [5]:
print('ABC'.encode('ascii'))
print('中文'.encode('utf-8'))
# print('中文'.encode('ascii'))

b'ABC'
b'\xe4\xb8\xad\xe6\x96\x87'


纯英文的str可以用ASCII编码为bytes，内容是一样的，含有中文的str可以用UTF-8编码为bytes。含有中文的str无法用ASCII编码，因为中文编码的范围超过了ASCII编码的范围，Python会报错。

在bytes中，无法显示为ASCII字符的字节，用\x##显示。

反过来，如果我们从网络或磁盘上读取了字节流，那么读到的数据就是bytes。要把bytes变为str，就需要用decode()方法：


In [6]:
print(b'ABC'.decode('ascii'))
print(b'\xe4\xb8\xad\xe6\x96\x87'.decode('utf-8'))

ABC
中文


如果bytes中包含无法解码的字节，decode()方法会报错：

In [7]:
print(b'\xe4\xb8\xad\xff'.decode('utf-8'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 3: invalid start byte

如果bytes中只有一小部分无效的字节，可以传入errors='ignore'忽略错误的字节：

In [8]:
print(b'\xe4\xb8\xad\xff'.decode('utf-8', errors='ignore'))

中


要计算str包含多少个字符，可以用len()函数：

In [9]:
print(len('ABC'))
print(len('中文'))

3
2


len()函数计算的是str的字符数，如果换成bytes，len()函数就计算字节数：

In [10]:
print(len(b'ABC'))
print(len(b'\xe4\xb8\xad\xe6\x96\x87'))
print(len('中文'.encode('utf-8')))

3
6
6


可见，1个中文字符经过UTF-8编码后通常会占用3个字节，而1个英文字符只占用1个字节。

在操作字符串时，我们经常遇到str和bytes的互相转换。为了避免乱码问题，应当始终坚持使用UTF-8编码对str和bytes进行转换。

由于Python源代码也是一个文本文件，所以，当你的源代码中包含中文的时候，在保存源代码时，就需要务必指定保存为UTF-8编码。当Python解释器读取源代码时，为了让它按UTF-8编码读取，我们通常在文件开头写上这两行：

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

第一行注释是为了告诉Linux/OS X系统，这是一个Python可执行程序，Windows系统会忽略这个注释；

第二行注释是为了告诉Python解释器，按照UTF-8编码读取源代码，否则，你在源代码中写的中文输出可能会有乱码。

申明了UTF-8编码并不意味着你的.py文件就是UTF-8编码的，必须并且要确保**文本编辑器正在使用UTF-8 without BOM编码**

如果.py文件本身使用UTF-8编码，并且也申明了# -*- coding: utf-8 -*-，打开命令提示符测试就可以正常显示中文

### 格式化

最后一个常见的问题是如何输出格式化的字符串。我们经常会输出类似'亲爱的xxx你好！你xx月的话费是xx，余额是xx'之类的字符串，而xxx的内容都是根据变量变化的，所以，需要一种简便的格式化字符串的方式。

在Python中，采用的格式化方式和C语言是一致的，用%实现，举例如下：


In [ ]:
print('Hello, %s' % 'world')
print('Hi, %s, you have $%d.' % ('Michael', 1000000))

你可能猜到了，%运算符就是用来格式化字符串的。在字符串内部，%s表示用字符串替换，%d表示用整数替换，有几个%?占位符，后面就跟几个变量或者值，顺序要对应好。如果只有一个%?，括号可以省略。

常见的占位符有：
%d 整数
%f 浮点数
%s 字符串
%x 十六进制整数

其中，格式化整数和浮点数还可以指定是否补0和整数与小数的位数：

In [11]:
print('%2d-%02d' % (3, 1))
print('%.2f' % 3.1415926)

 3-01
3.14


如果你不太确定应该用什么，%s永远起作用，它会把任何数据类型转换为字符串：

In [12]:
print('Age: %s. Gender: %s' % (25, True))

Age: 25. Gender: True


有些时候，字符串里面的%是一个普通字符怎么办？这个时候就需要转义，用%%来表示一个%：

In [13]:
print('growth rate: %d %%' % 7)

growth rate: 7 %


### format()

另一种格式化字符串的方法是使用字符串的format()方法，它会用传入的参数依次替换字符串内的占位符{0}、{1}……，不过这种方式写起来比%要麻烦得多：


In [14]:
print('Hello, {0}, 成绩提升了 {1:.1f}%'.format('小明', 17.125))

Hello, 小明, 成绩提升了 17.1%


### f-string

最后一种格式化字符串的方法是使用以f开头的字符串，称之为f-string，它和普通字符串不同之处在于，字符串如果包含{xxx}，就会以对应的变量替换

In [15]:
r = 2.5
s = 3.14 * r ** 2
print(f'The area of a circle with radius {r} is {s:.2f}')

The area of a circle with radius 2.5 is 19.62


上述代码中，{r}被变量r的值替换，{s:.2f}被变量s的值替换，并且:后面的.2f指定了格式化参数（即保留两位小数），因此，{s:.2f}的替换结果是19.62

## list

Python内置的一种数据类型是列表：list。list是一种有序的集合，可以随时添加和删除其中的元素。

比如，列出班里所有同学的名字，就可以用一个list表示：

In [17]:
classmates = ['Michael', 'Bob', 'Tracy']
print(classmates)

['Michael', 'Bob', 'Tracy']


变量classmates就是一个list。用len()函数可以获得list元素的个数

In [18]:
print(len(classmates))

3


用索引来访问list中每一个位置的元素，记得索引是从0开始的：

In [19]:
print(classmates[0])
print(classmates[1])
print(classmates[2])

Michael
Bob
Tracy


当索引超出了范围时，Python会报一个IndexError错误，所以，要确保索引不要越界，记得最后一个元素的索引是len(classmates) - 1。

如果要取最后一个元素，除了计算索引位置外，还可以用-1做索引，直接获取最后一个元素：

In [20]:
print(classmates[-1])

Tracy


In [21]:
print(classmates[-2])
print(classmates[-3])

Bob
Michael


list是一个可变的有序表，所以，可以往list中追加元素到末尾：

In [22]:
classmates.append('Adam')
classmates.append('Alex')
print(classmates)

['Michael', 'Bob', 'Tracy', 'Adam', 'Alex']


也可以把元素插入到指定的位置，比如索引号为1的位置：

In [23]:
classmates.insert(1, 'Jack')
print(classmates)

['Michael', 'Jack', 'Bob', 'Tracy', 'Adam', 'Alex']


要删除list末尾的元素，用pop()方法;
要删除指定位置的元素，用pop(i)方法，其中i是索引位置;

In [25]:
classmates.pop()
classmates.pop(1)
print(classmates)

['Michael', 'Tracy']


要把某个元素替换成别的元素，可以直接赋值给对应的索引位置：

In [26]:
classmates[1] = 'Sarah'
print(classmates)

['Michael', 'Sarah']


list里面的元素的数据类型也可以不同，比如：

In [ ]:
L = ['Apple', 123, True]

list元素也可以是另一个list，比如：

In [27]:
s = ['python', 'java', ['asp', 'php'], 'scheme']
# 要注意s只有4个元素，其中s[2]又是一个list，如果拆开写就更容易理解了
# p = ['asp', 'php']
# s = ['python', 'java', p, 'scheme']

print(len(s))

4


要拿到'php'可以写p[1]或者s[2][1]，因此s可以看成是一个二维数组，类似的还有三维、四维……数组，不过很少用到。

如果一个list中一个元素也没有，就是一个空的list，它的长度为0：

In [28]:
L = []
print(len(L))

0


## tuple

另一种有序列表叫元组：tuple。tuple和list非常类似，但是tuple一旦初始化就不能修改，比如同样是列出同学的名字：

In [ ]:
classmates = ('Michael', 'Bob', 'Tracy')

现在，classmates这个tuple不能变了，它也没有append()，insert()这样的方法。其他获取元素的方法和list是一样的，你可以正常地使用classmates[0]，classmates[-1]，但不能赋值成另外的元素。

不可变的tuple有什么意义？因为tuple不可变，所以代码更安全。如果可能，能用tuple代替list就尽量用tuple。

tuple的陷阱：当你定义一个tuple时，在定义的时候，tuple的元素就必须被确定下来，比如：

In [30]:
t = (1, 2)
t

(1, 2)

如果要定义一个空的tuple，可以写成()：

In [31]:
t = ()
t

()

但是，要定义一个只有1个元素的tuple，如果你这么定义：

In [32]:
t = (1)
t

1

定义的不是tuple，是1这个数！这是因为括号()既可以表示tuple，又可以表示数学公式中的小括号，这就产生了歧义，因此，Python规定，这种情况下，按小括号进行计算，计算结果自然是1。

**所以，只有1个元素的tuple定义时必须加一个逗号,来消除歧义：**

In [33]:
t = (1, )
t

(1,)

Python在显示只有1个元素的tuple时，也会加一个逗号,，以免你误解成数学计算意义上的括号。

最后来看一个“可变的”tuple：

In [34]:
t = ('a', 'b', ['A', 'B'])
t[2][0] = 'X'
t[2][1] = 'Y'
t

('a', 'b', ['X', 'Y'])

这个tuple定义的时候有3个元素，分别是'a'，'b'和一个list。不是说tuple一旦定义后就不可变了吗？怎么后来又变了？

tuple的元素确实变了，但其实变的不是tuple的元素，而是list的元素。tuple一开始指向的list并没有改成别的list，所以，tuple所谓的“不变”是说，tuple的每个元素，指向永远不变。即指向'a'，就不能改成指向'b'，指向一个list，就不能改成指向其他对象，但指向的这个list本身是可变的！

理解了“指向不变”后，要创建一个内容也不变的tuple怎么做？那就必须保证tuple的每一个元素本身也不能变。

## 条件判断

if判断条件还可以简写，比如写：
只要x是非零数值、非空字符串、非空list等，就判断为True，否则为False

In [ ]:
if x:
    print('True')

### input

很多同学会用input()读取用户的输入，这样可以自己输入，程序运行得更有意思：

In [ ]:
s = input('birth: ')
birth = int(s)
if birth < 2000:
    print('00前')
else:
    print('00后')

### 模式匹配

当我们用if ... elif ... elif ... else ...判断时，会写很长一串代码，可读性较差。

如果要针对某个变量匹配若干种情况，可以使用match语句。

例如，某个学生的成绩只能是A、B、C，用if语句编写如下：

In [ ]:
score = 'B'
if score == 'A':
    print('score is A.')
elif score == 'B':
    print('score is B.')
elif score == 'C':
    print('score is C.')
else:
    print('invalid score.')

如果用match语句改写，则改写如下：

In [41]:
score = 'B'

match score:
    case 'A':
        print('score is A.')
    case 'B':
        print('score is B.')
    case 'C':
        print('score is C.')
    case _:
        print('invalid score.')


SyntaxError: invalid syntax (805763417.py, line 3)

使用match语句时，我们依次用case xxx匹配，并且可以在最后（且仅能在最后）加一个case _表示“任意值”，代码较if ... elif ... else ...更易读。

### 复杂匹配

match语句除了可以匹配简单的单个值外，还可以匹配多个值、匹配一定范围，并且把匹配后的值绑定到变量：

In [44]:
age = 9

match age:
    case x if x < 10:
        print(f'< 10 years old: {x}')
        """
        f 字符串（也称为 f-strings 或格式化字符串字面值）是一种特殊的字符串，允许在字符串中嵌入变量。通过在字符串前加上 f 或 F，您可以在字符串中使用花括号 {} 来包含变量，而这些变量的值会在运行时插入到字符串中。

        例如，代码中的 f'< 10 years old: {x}' 是一个 f-string，其中 {x} 将会被变量 x 的值替换。这种语法使得字符串的构建变得更加简洁和可读。
        """
    case 10:
        print('10 years old.')
    case 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18:
        print('11~18 years old.')
    case 19:
        print('19 years old.')
    case _:
        print('not sure.')

SyntaxError: invalid syntax (1622765496.py, line 3)

在上面这个示例中，第一个case x if x < 10表示当age < 10成立时匹配，且赋值给变量x，第二个case 10仅匹配单个值，第三个case 11|12|...|18能匹配多个值，用|分隔。

可见，match语句的case匹配非常灵活。

### 匹配列表

match语句还可以匹配列表，功能非常强大。

我们假设用户输入了一个命令，用args = ['gcc', 'hello.c']存储，下面的代码演示了如何用match匹配来解析这个列表：

In [45]:
args = ['gcc', 'hello.c', 'world.c']
# args = ['clean']
# args = ['gcc']

match args:
    # 如果仅出现gcc，报错:
    case ['gcc']:
        print('gcc: missing source file(s).')
    # 出现gcc，且至少指定了一个文件:
    case ['gcc', file1, *files]:
        print('gcc compile: ' + file1 + ', ' + ', '.join(files))
    # 仅出现clean:
    case ['clean']:
        print('clean')
    case _:
        print('invalid command.')


SyntaxError: invalid syntax (2130931557.py, line 5)

第一个case ['gcc']表示列表仅有'gcc'一个字符串，没有指定文件名，报错；

第二个case ['gcc', file1, *files]表示列表第一个字符串是'gcc'，第二个字符串绑定到变量file1，后面的任意个字符串绑定到*files（符号作用将在函数的参数中讲解），它实际上表示至少指定一个文件；

第三个case ['clean']表示列表仅有'clean'一个字符串；

最后一个case _表示其他所有情况。

可见，match语句的匹配规则非常灵活，可以写出非常简洁的代码。

## 循环

Python的循环有两种，一种是for...in循环，依次把list或tuple中的每个元素迭代出来，看例子：

In [46]:
names = ['Michael', 'Bob', 'Tracy']
for name in names:
    print(name)

Michael
Bob
Tracy


所以for x in ...循环就是把每个元素代入变量x，然后执行缩进块的语句。

再比如我们想计算1-10的整数之和，可以用一个sum变量做累加：

In [47]:
sum = 0
for x in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    sum = sum + x
print(sum)

55


如果要计算1-100的整数之和，从1写到100有点困难，幸好Python提供一个range()函数，可以生成一个整数序列，再通过list()函数可以转换为list。比如range(5)生成的序列是从0开始小于5的整数：

In [48]:
print(list(range(5)))

[0, 1, 2, 3, 4]


range(101)就可以生成0-100的整数序列，计算如下：

In [ ]:
sum = 0
for x in range(101):
    sum = sum + x
print(sum)

第二种循环是while循环，只要条件满足，就不断循环，条件不满足时退出循环。比如我们要计算100以内所有奇数之和，可以用while循环实现：

In [49]:
sum = 0
n = 99
while n > 0:
    sum += n
    n -= 2
print(sum)

2500


### break

In [50]:
n = 1
while n <= 100:
    if n > 10: # 当n = 11时，条件满足，执行break语句
        break # break语句会结束当前循环
    print(n)
    n = n + 1
print('END')

1
2
3
4
5
6
7
8
9
10
END


### continue

In [ ]:
n = 0
while n < 10:
    n = n + 1
    if n % 2 == 0: # 如果n是偶数，执行continue语句
        continue # continue语句会直接继续下一轮循环，后续的print()语句不会执行
    print(n)

## dict

Python内置了字典：dict的支持，dict全称dictionary，在其他语言中也称为map，使用键-值（key-value）存储，具有极快的查找速度。

举个例子，假设要根据同学的名字查找对应的成绩，如果用list实现，需要两个list：

In [ ]:
names = ['Michael', 'Bob', 'Tracy']
scores = [95, 75, 85]

给定一个名字，要查找对应的成绩，就先要在names中找到对应的位置，再从scores取出对应的成绩，list越长，耗时越长。

如果用dict实现，只需要一个“名字”-“成绩”的对照表，直接根据名字查找成绩，无论这个表有多大，查找速度都不会变慢。用Python写一个dict如下：

In [54]:
d = {'Michael': 95, 'Bob': 75, 'Tracy': 85}
print(d['Michael'])

95


为什么dict查找速度这么快？因为dict的实现原理和查字典是一样的。假设字典包含了1万个汉字，我们要查某一个字，一个办法是把字典从第一页往后翻，直到找到我们想要的字为止，这种方法就是在list中查找元素的方法，list越大，查找越慢。

第二种方法是先在字典的索引表里（比如部首表）查这个字对应的页码，然后直接翻到该页，找到这个字。无论找哪个字，这种查找速度都非常快，不会随着字典大小的增加而变慢。

dict就是第二种实现方式，给定一个名字，比如'Michael'，dict在内部就可以直接计算出Michael对应的存放成绩的“页码”，也就是95这个数字存放的内存地址，直接取出来，所以速度非常快。

你可以猜到，这种key-value存储方式，在放进去的时候，必须根据key算出value的存放位置，这样，取的时候才能根据key直接拿到value。

把数据放入dict的方法，除了初始化时指定外，还可以通过key放入：

In [55]:
d['Adam'] = 67
print(d)

{'Michael': 95, 'Bob': 75, 'Tracy': 85, 'Adam': 67}


由于一个key只能对应一个value，所以，多次对一个key放入value，后面的值会把前面的值冲掉：

In [56]:
d['Jack'] = 90
print(d['Jack'])
d['Jack'] = 83
print(d['Jack'])

90
83


如果key不存在，dict就会报错：

In [57]:
print(d['Thomax'])

KeyError: 'Thomax'

要避免key不存在的错误，有两种办法，一是通过in判断key是否存在：

In [59]:
print('Thomas' in d)

False


二是通过dict提供的get()方法，如果key不存在，可以返回None，或者自己指定的value：

In [61]:
print(d.get('Thomas'))
print(d.get('Thomas', -1))

None
-1


注意：返回None的时候Python的交互环境不显示结果。

要删除一个key，用pop(key)方法，对应的value也会从dict中删除：

In [62]:
d.pop('Bob')
print(d)

{'Michael': 95, 'Tracy': 85, 'Adam': 67, 'Jack': 83}


请务必注意，dict内部存放的顺序和key放入的顺序是没有关系的。

和list比较，dict有以下几个特点：

查找和插入的速度极快，不会随着key的增加而变慢；
需要占用大量的内存，内存浪费多。
而list相反：

查找和插入的时间随着元素的增加而增加；
占用空间小，浪费内存很少。
所以，dict是用空间来换取时间的一种方法。

dict可以用在需要高速查找的很多地方，在Python代码中几乎无处不在，正确使用dict非常重要，需要牢记的第一条就是dict的key必须是不可变对象。

这是因为dict根据key来计算value的存储位置，如果每次计算相同的key得出的结果不同，那dict内部就完全混乱了。这个通过key计算位置的算法称为哈希算法（Hash）。

要保证hash的正确性，作为key的对象就不能变。在Python中，字符串、整数等都是不可变的，因此，可以放心地作为key。而list是可变的，就不能作为key：

In [63]:
key = [1, 2, 3]
d[key] = 'a list'

TypeError: unhashable type: 'list'

## set

set和dict类似，也是一组key的集合，但不存储value。由于key不能重复，所以，*在set中，没有重复的key。*

要创建一个set，需要提供一个list作为输入集合：

In [64]:
s = set([1, 2, 3])
print(s)

{1, 2, 3}


注意，传入的参数[1, 2, 3]是一个list，而显示的{1, 2, 3}只是告诉你这个set内部有1，2，3这3个元素，显示的顺序也不表示set是有序的。。

重复元素在set中自动被过滤：

In [65]:
s = set([1, 1, 2, 2, 3, 3])
print(s)

{1, 2, 3}


通过add(key)方法可以添加元素到set中，可以重复添加，但不会有效果：

In [66]:
s.add(4)
s.add(5)
s.add(6)

print(s)

{1, 2, 3, 4, 5, 6}


通过remove(key)方法可以删除元素：

In [67]:
s.remove(4)
print(s)

{1, 2, 3, 5, 6}


set可以看成数学意义上的无序和无重复元素的集合，因此，两个set可以做数学意义上的交集、并集等操作：

In [71]:
s1 = set([1, 2, 3])
s2 = set([2, 3, 4])
print(s1 & s2)
print(s1 | s2)

{2, 3}
{1, 2, 3, 4}


set和dict的唯一区别仅在于没有存储对应的value，但是，set的原理和dict一样，所以，同样不可以放入可变对象，因为无法判断两个可变对象是否相等，也就无法保证set内部“不会有重复元素”。

## 不可变对象

上面我们讲了，str是不变对象，而list是可变对象。

对于可变对象，比如list，对list进行操作，list内部的内容是会变化的，比如：

In [72]:
a = ['c', 'b', 'a']
a.sort()
print(a)

['a', 'b', 'c']


而对于不可变对象，比如str，对str进行操作呢：

In [74]:
a = 'abc'
print(a.replace('a', 'A'))
print(a)

Abc
abc


虽然字符串有个replace()方法，也确实变出了'Abc'，但变量a最后仍是'abc'，应该怎么理解呢？

我们先把代码改成下面这样：

In [75]:
a = 'abc'
b = a.replace('a', 'A')
print(b)
print(a)

Abc
abc


要始终牢记的是，a是变量，而'abc'才是字符串对象！有些时候，我们经常说，对象a的内容是'abc'，但其实是指，a本身是一个变量，它指向的对象的内容才是'abc';

当我们调用a.replace('a', 'A')时，实际上调用方法replace是作用在字符串对象'abc'上的，而这个方法虽然名字叫replace，但却没有改变字符串'abc'的内容。相反，replace方法创建了一个新字符串'Abc'并返回，如果我们用变量b指向该新字符串，就容易理解了，变量a仍指向原有的字符串'abc'，但变量b却指向新字符串'Abc'了;

所以，对于不变对象来说，调用对象自身的任意方法，也不会改变该对象自身的内容。相反，这些方法会创建新的对象并返回，这样，就保证了不可变对象本身永远是不可变的。

选择不可变对象作为key很重要，最常用的key是字符串

将(1, 2, 3)和(1, [2, 3])放入dict（字典）或set（集合）时，会有一些不同的行为和结果。首先，我们来看一下两者的差异：

In [76]:
# Attempting to create a dictionary with tuple keys
my_dict = {(1, 2, 3): 'Tuple 1'}
print(my_dict)

my_dict = {(1, [2, 3]): 'Tuple 2'}
print(my_dict)


{(1, 2, 3): 'Tuple 1'}


TypeError: unhashable type: 'list'

In [77]:
# Attempting to create a set with tuples
my_set = {(1, 2, 3)}
print(my_set)

my_set = {(1, [2, 3])}
print(my_set)

{(1, 2, 3)}


TypeError: unhashable type: 'list'

如果想要在集合或字典中包含类似的元素，可以使用不可变类型，例如字符串、整数或其他不可变类型。如果需要使用可变类型，可能需要考虑使用其他数据结构或者在需要时使用特殊处理的方法。

如果需要在字典或集合中使用可变类型，可以考虑使用元组作为字典的键，但是确保元组中的所有元素都是不可变的。对于集合，元素必须是不可变的。

下面是一个例子，其中使用包含可变类型的元组放入字典：

In [ ]:
# Using tuples with immutable elements in a dictionary
my_dict = {(1, 2, 3): 'Tuple 1', (1, tuple([2, 3])): 'Tuple 2'}
print(my_dict)


在这个例子中，我们将列表 `[2, 3]` 转换为元组，因为元组是不可变的，从而避免了 `TypeError`。

如果希望在集合中使用可变类型，您可以将列表转换为元组：

In [ ]:
# Using tuples with immutable elements in a set
my_set = {(1, 2, 3), tuple([1, 2, 3])}
print(my_set)

同样，我们在集合中使用了包含可变类型的元组。这样，就可以成功创建包含可变类型的元素的字典或集合。

- `list`（列表）是一种有序的可变序列，可以存储任意类型的元素。列表使用方括号`[]`来表示，元素之间用逗号`,`分隔。列表支持索引、切片、添加、删除、修改等操作，是Python中最常用的数据类型之一。

- `tuple`（元组）是一种有序的不可变序列，可以存储任意类型的元素。元组使用圆括号`()`来表示，元素之间用逗号`,`分隔。元组支持索引、切片等操作，但不支持添加、删除、修改等操作。元组通常用于存储不可变的数据，如坐标、颜色等。

- `dict`（字典）是一种无序的键值对集合，可以存储任意类型的键和值。字典使用花括号`{}`来表示，每个键值对之间用冒号`:`分隔，键值对之间用逗号`,`分隔。字典支持通过键来访问值，也支持添加、删除、修改等操作。字典通常用于存储具有映射关系的数据，如姓名和电话号码的对应关系。

- `set`（集合）是一种无序的元素集合，可以存储任意类型的元素。集合使用花括号`{}`来表示，元素之间用逗号`,`分隔。集合支持添加、删除、交集、并集、差集等操作。集合通常用于去重、交集、并集等操作。

需要注意的是，`list`、`tuple`、`dict`和`set`是不同的数据类型，它们之间不能直接进行转换。如果需要将它们之间进行转换，需要使用相应的转换函数，如`list()`、`tuple()`、`dict()`和`set()`。

## 速查

list是方括号[ ]，如L=[1,2,3]

tuple是圆括号( )，如t=(1,2,3)

dict是花括号{ }，如d={'张三':1, ’王五':2, '赵六':3}

set也是花括号{ }，如s={1,2,3}

增删改查：

1、list

1）查: L[1] L[0]      L[-1]  #索引最后一个元素

2）增: 
L.append('赵六') #添加到最后一个位置    
L.insert(2,'赵六') #添加到指定位置

3）删: 
L.pop() #删除最后一个位置的元素     
L.pop(1) #删除指定位置的元素

4）改: L[1]='王丹'

5）空: L=[ ]

2、Tuple：

1）不能删改内容，索引方法和list相同

2）定义只有一个元素的tulpe： t=('张三',)

3、dict：

1）查：'李四' in d #查是否存在     
d.get('李四') #不存在则无反馈 或d.get('李四',-1) #不存在返回-1

2）增/改：d['李四']=80

3）删：d.pop('李四')

4、set（一组key的合集，无value，不重复，无序）

1）增：s.add(4)

2）删：s.remove(4)

3）交集：s1&s2

2）并集：s1 I s2